# Graph convolutional networks

Resources:

- https://towardsdatascience.com/how-to-do-deep-learning-on-graphs-with-graph-convolutional-networks-7d2250723780
- https://github.com/tkipf/pygcn
- http://tkipf.github.io/graph-convolutional-networks/

## Setup

In [328]:
import math

import numpy as np
import scipy.sparse as sp
from networkx import karate_club_graph, to_numpy_matrix
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelBinarizer
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module

## Dataset

In [391]:
# Load the dataset
zkc = karate_club_graph()
order = sorted(list(zkc.nodes()))
labels = [zkc.nodes[i]["club"] for i in order]
labels = LabelBinarizer().fit_transform(y=labels)
labels = labels.reshape(-1)

# Build the adjency matrix A
A = to_numpy_matrix(zkc, nodelist=order)  # 34x34 shape

## Pytorch Model

### Utils

In [374]:
def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    
    return mx

def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    
    return correct / len(labels)

### Layers

In [263]:
class GraphConvolution(Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter("bias", None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, X, adj):
        support = torch.mm(X, self.weight)
        output = torch.spmm(adj, support)

        if self.bias is not None:
            output = output + self.bias
        
        return output

    def __repr__(self):
        return "{0} ({1} -> {2})".format(self.__class__.__name__,
                                         self.in_features,
                                         self.out_features)

### Model

In [372]:
class GCN(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout):
        super(GCN, self).__init__()

        self.gc1 = GraphConvolution(nfeat, nhid)
        self.gc2 = GraphConvolution(nhid, nclass)
        self.dropout = dropout

    def forward(self, x, adj):
        x = F.relu(self.gc1(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gc2(x, adj)
        
        return F.log_softmax(x, dim=1)

### Training

#### Prepare training data

In [365]:
# Add identity matrix --> A_hat
assert np.allclose(np.asarray(A.diagonal()), np.zeros(A.shape[0]))
A_hat = A + sp.eye(A.shape[0])
assert np.allclose(np.asarray(A_hat.diagonal()), np.ones(A_hat.shape[0]))

# Normalize A_hat
A_hat_norm = normalize(A_hat)
assert A_hat_norm.shape == A.shape
assert np.allclose(np.asarray(A_hat_norm.sum(axis=1)), np.ones((A.shape[0], 1)))

In [366]:
# As we have no features, use the identity matrix
features = np.eye(A.shape[0])
features = normalize(features)  # useless for now

In [392]:
# labels
labels = torch.LongTensor(labels)

In [393]:
# Convert to torch tensor
A_hat_norm = torch.FloatTensor(np.asarray(A_hat_norm))
features = torch.FloatTensor(features)

#### Actual training

In [423]:
model = GCN(nfeat=features.shape[1],
            nhid=4,
            nclass=2,
            dropout=.3)

optimizer = optim.Adam(model.parameters(),
                       lr=.05,
                       weight_decay=.01)

def train(epoch):
    model.train()
    optimizer.zero_grad()
    output = model(features, A_hat_norm)

    loss_train = F.nll_loss(output, labels)
    acc_train = accuracy(output, labels)
    loss_train.backward()
    optimizer.step()
    
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()))

for i in range(15):
    train(i)

Epoch: 0001 loss_train: 0.7618 acc_train: 0.5000
Epoch: 0002 loss_train: 0.7331 acc_train: 0.5000
Epoch: 0003 loss_train: 0.7069 acc_train: 0.5000
Epoch: 0004 loss_train: 0.6949 acc_train: 0.5000
Epoch: 0005 loss_train: 0.6782 acc_train: 0.5000
Epoch: 0006 loss_train: 0.6620 acc_train: 0.5000
Epoch: 0007 loss_train: 0.6437 acc_train: 0.5000
Epoch: 0008 loss_train: 0.6361 acc_train: 0.5000
Epoch: 0009 loss_train: 0.6239 acc_train: 0.6765
Epoch: 0010 loss_train: 0.5865 acc_train: 0.8824
Epoch: 0011 loss_train: 0.5548 acc_train: 0.9412
Epoch: 0012 loss_train: 0.5333 acc_train: 0.9118
Epoch: 0013 loss_train: 0.5022 acc_train: 0.9706
Epoch: 0014 loss_train: 0.5458 acc_train: 0.9412
Epoch: 0015 loss_train: 0.4635 acc_train: 0.9412
